In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from matplotlib import font_manager, rc
import datetime

In [ ]:
pollution = pd.read_csv('/kaggle/input/seoulairreport/SeoulHourlyAvgAirPollution.csv')
pollution.columns = ['date','state','co2','ozone','co','so2','pm10','pm2.5']
pollution.head(3)

# 프로젝트 선언
* 위의 데이터는 서울 환경부에서 제공하고, 열린 데이터광장에서 배포하는 데이터이다.
* 각 기간에 따른 서울의 구별 환경 오염의 실태를 담고있다.
* 서울은 한반도의 20~30%의 면적을 자치한다고 나와있기에, 수도인 서울의 대기 오염 상태가 대한민국의 대기오염 상태를 대변한다 해도 과언이 아니다.
* 이런 이유를 통해서 이번 데이터 분석을 통해, 서울의 대기오염 실태를 파악하고, 대기오염의 각 원인별 상관성, 시계
열분석을 통해 기간별 환경문제를 파악한다
* 또한 구별 대기오염 실태를 순위로 파악하여 대기오염 수준이 가장 나쁜 4지역을 추출한다. 이를 통해 해당 지역에 패널티를 부여하여 대기오염을 개선하기 위한 프로젝트를 시작한다.

In [ ]:
pollution.shape

In [ ]:
pollution.info()

* 결측치가 존재한다.
* 결측치를 제거하거나 대치해준다.
    * 해당 분석에 있어서는, 모델을 생성하거나, 서비스를 구현하는 목적이 아닌, 실태파악에 있음으로 머신러닝을 통한 결측치 대체는 진행하지 않도록 한다


In [ ]:
pollution=pollution.fillna(method='ffill')

* 해당 데이터는 기간별로 구분되어 있다. 따라서 하루하루마다의 수치들은 큰 차이가 없을 것으로 판단, 이전 값으로 대치해주는 선택을 진행했다.

# 데이터 분포 파악

In [ ]:
sns.pairplot(pollution)

# 서울 지역별 분석
* 서울의 25지역내의 관측소를 기준으로 오염 농도를 파악한것으로 판단된다.
    * 지역 모두 169개의 값을 갖고 있음으로 편향의 문제는 제기하지 않도록 한다.

In [ ]:
pm10_score=pollution.groupby(pollution.state)['pm10'].mean()
pm10_score=pm10_score.sort_values(ascending=False)
pm2_5_score = pollution.groupby(pollution.state)['pm2.5'].mean()
pm2_5_score = pm2_5_score.sort_values(ascending=False)

In [ ]:
rc('font',family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False
plt.figure(figsize=(10,6))
plt.subplot(2,1,1) # 2행 1열로 나타낼거고 지금은 그중에 1째 그래프를 그리겠다.
plt.title('미세먼지')
plt.xticks(rotation=45)
plt.bar(pm10_score.index,pm10_score.values,color='gold')
plt.subplot(2,1,2)
plt.title('초미세먼지')
plt.xticks(rotation=45)
plt.bar(pm2_5_score.index,pm2_5_score.values,color='brown')
plt.tight_layout()

* 미세먼지의 농도에 따라서 반드시 초 미세 먼지의 농도도 높아진다는 것은 확인할 수 없었다.
* 단 양천구는 초미세먼지, 미세먼지의 농도가 모두 1등을 차지할 정도로 대기오염이 심했다. 

# 그렇다면 실제로 양천구가 우리나라 평균 미세먼지 농도보다 높은 지역일까?
 * 가설검정

In [ ]:
pm10_state=pollution.groupby(pollution.state)['pm10'].mean()
pm10_state[pm10_state.index=='양천구']

In [ ]:
pm10_t = pollution.pm10
t_alpha, p_value= stats.ttest_1samp(pollution.pm10,35)
print('t statistic : %.3f \np-value : %.10f' % (t_alpha,p_value))

* 그래프를 통해서 양천구가 미세먼지, 초미세먼지가 가장 나쁜 것을 알 수 있었지만 가설검정을 통해 실제로 유의미하게 차이가 있는지 확인을 했다.
* ttest의 결과로 t값은 4.8 p_values는 0.000013으로 평균의 차이가 있음을 확인했다.

* 관측기간 확인

In [ ]:
pollution.date=pollution.date.astype('str')
pollution.date=pollution.date.str[:-4]
date_obj=pd.to_datetime(pollution.date,yearfirst=True)
#pollution.date.strptime(date_string,'%Y-%m-%d')
pollution['date_obj'] = date_obj
print(f'최대기간:{date_obj.max()},최소기간:{date_obj.min()}')
print(f'관측기간:{date_obj.max()-date_obj.min()}')

# 시게열분석

In [ ]:
plt.figure(figsize=(9,7))
sns.lineplot(pollution.date_obj,pollution.pm10,label='미세먼지')
plt.xticks(rotation=45)
sns.lineplot(pollution.date_obj,pollution['pm2.5'],label='초미세먼지')

# 상관분석

In [ ]:
corr = pollution.drop(['date','date_obj'],axis=1).corr()
corr